# 第6章: 単語ベクトル

## 50. 単語ベクトルの読み込みと表示

In [10]:
# 学習済み単語ベクトルのダウンロード
!pip install gdown
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM -O data/GoogleNews-vectors-negative300.bin.gz

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

but Gdown can't. Please check connections and permissions.


In [19]:
!pip install gensim

In [20]:
from gensim.models import KeyedVectors
model_path = "./data/GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(model_path, binary=True)
model["United_States"]

# 単語ベクトルの表示方法  https://qiita.com/nymwa/items/9963968568b9dbfc2e78

ModuleNotFoundError: No module named 'numpy.rec'

## 51. 単語の類似度

In [21]:
print(model.similarity("United_States", "U.S."))

# コサイン類似度の計算 https://qiita.com/DancingEnginee1/items/b10c8ef7893d99aa53be

NameError: name 'model' is not defined

## 52. 類似度の高い単語10件

In [22]:
import numpy as np
import pandas as pd

pd.DataFrame(
  model.most_similar("United_States"),
  columns=["単語", "類似度"],
  index=np.arange(10) + 1
)

# pandasの使い方        https://qiita.com/shimotaroo/items/d3fd17b4d5f470cae957
# 連番の生成            https://deepage.net/features/numpy-arange.html#%E3%81%82%E3%82%8B%E7%B5%82%E7%82%B9%E3%81%A0%E3%81%91%E3%82%92%E6%8C%87%E5%AE%9A%E3%81%97%E3%81%9F%E5%A0%B4%E5%90%88

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## 53. 加法構成性によるアナロジー

In [23]:
pd.DataFrame(
  model.most_similar(positive=["Spain", "Athens"], negative="Madrid"),
  columns=["単語", "類似度"],
  index=np.arange(10) + 1
)

NameError: name 'pd' is not defined

## 54. アナロジーデータでの実験

In [24]:
# 単語アナロジーの評価データのダウンロード
!wget http://download.tensorflow.org/data/questions-words.txt -P data/

--2025-04-18 11:56:40--  http://download.tensorflow.org/data/questions-words.txt
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.31.207, 142.251.111.207, 142.251.16.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.31.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘data/questions-words.txt’

questions-words.txt 100%[===================>] 589.80K  --.-KB/s    in 0.002s  

2025-04-18 11:56:40 (264 MB/s) - ‘data/questions-words.txt’ saved [603955/603955]



In [25]:
# dataframeの作成
data = []
with open("./data/questions-words.txt") as f:
  for line in f:
    if line.startswith(":"):
      section = line[1:].strip()
    else:
      words = line.split()
      if len(words) == 4:
        data.append([section] + words)
df = pd.DataFrame(data, columns=["section", "1", "2", "3", "4"])

# 類似度が最も高い単語とその類似度の計算
similarity_words = []
similarities = []
capital_df = df[df["section"] == "capital-common-countries"]
for _, row in capital_df.iterrows():
  vec = model[row["2"]] - model[row["1"]] + model[row["3"]]
  word, similarity = model.similar_by_vector(vec, 1)[0]
  similarity_words.append(word)
  similarities.append(similarity)

capital_df["5"] = similarity_words
capital_df["類似度"] = similarities
print(capital_df)

NameError: name 'pd' is not defined

## 55. アナロジータスクでの正解率

In [26]:
correct = (capital_df["4"] == capital_df["5"])
acc = correct.sum() / len(correct)
print(f"{acc:.2%}")

# dataframeの比較 https://pythonmaniac.com/compare-dataframe-contents/#index_id2

NameError: name 'capital_df' is not defined

## 56. WordSimilarity-353での評価

In [27]:
# The WordSimilarity-353 Test Collectionの評価データをダウンロード
!wget https://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip -P data/

--2025-04-18 11:56:43--  https://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip
Resolving www.gabrilovich.com (www.gabrilovich.com)... 173.236.137.139
Connecting to www.gabrilovich.com (www.gabrilovich.com)|173.236.137.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23257 (23K) [application/zip]
Saving to: ‘data/wordsim353.zip.2’

wordsim353.zip.2    100%[===================>]  22.71K  --.-KB/s    in 0.001s  

2025-04-18 11:56:43 (15.0 MB/s) - ‘data/wordsim353.zip.2’ saved [23257/23257]



In [28]:
import zipfile

with zipfile.ZipFile("./data/wordsim353.zip", "r") as zf:
  with zf.open("combined.csv") as f:
    df = pd.read_csv(f)
print(df.head())

NameError: name 'pd' is not defined

## 57. k-meansクラスタリング

## 58. Ward法によるクラスタリング

## 59. t-SNEによる可視化